In [1]:
import copy
from fireplace.exceptions import GameOver
from fireplace.utils import play_full_game
import logging
import random
from fireplace import cards
import pstats

%load_ext snakeviz

cards.db.initialize()
logging.disable(logging.WARNING) #logging.NOTSET

[fireplace.__init__]: Initializing card database
INFO:fireplace:Initializing card database
[fireplace.__init__]: Merged 2253 cards
INFO:fireplace:Merged 2253 cards


In [2]:
def run_game():
    try:
        play_full_game();
    except GameOver:
        pass

In [3]:
%%capture c
random.seed(1857)
%timeit -n100 run_game()

In [4]:
c.stdout.splitlines()[-1]

'100 loops, best of 3: 121 ms per loop'

In [5]:
%%capture
%%prun -D fireplace.prof
random.seed(1857)
for _ in range(100):
    run_game()

In [9]:
stats_f = 'fireplace.prof'
stats = pstats.Stats(stats_f)
stats.strip_dirs()
stats.dump_stats(stats_f)

In [42]:
import pandas as pd
from io import StringIO

index = pd.MultiIndex.from_tuples(stats.stats.keys())
df = pd.DataFrame(list(stats.stats.values()), index=index)
df.drop(df.columns[[1, 4]], axis=1, inplace=True)
df.columns = ['ncalls', 'tottime', 'cumtime']
df.index.rename(['filename', 'lineno', 'function'], inplace=True)
df

,,,ncalls,tottime,cumtime
filename,lineno,function,,,
actions.py,416,do,236,0.001663,0.268884
card.py,432,attack_targets,12706,0.050073,0.753553
game.py,37,__iter__,42376,0.105575,0.964367
actions.py,9,_eval_card,783,0.002931,0.077295
__init__.py,1281,warning,75,0.000046,0.000079
card.py,565,__init__,5948,0.082938,0.383074
actions.py,235,do,25,0.000046,0.001704
managers.py,58,action_start,19907,0.010379,0.010379
actions.py,413,get_args,236,0.000428,0.000488


In [58]:
pd.options.display.float_format = '{:.2f}'.format
c = 'tottime'
pd.DataFrame(df.groupby(level='filename').sum()[c]  / df[c].sum()) \
    .sort_values(by=c, ascending=False)

,tottime
filename,
card.py,0.16
~,0.15
entity.py,0.13
selector.py,0.10
actions.py,0.10
game.py,0.09
cardxml.py,0.06
player.py,0.05
utils.py,0.04
